# 000 Forecasting Bot

Starting from https://colab.research.google.com/drive/1_Il5h2Ed4zFa6Z3bROVCE68LZcSi4wHX?usp=sharing

## Imports

In [1]:
from IPython.display import Markdown

## Today

In [2]:
import datetime
today = str(datetime.datetime.now())[0:10]
today

'2024-08-01'

## 000_bot

### API Keys

In order to run this notebook as is, you'll need to enter a few API keys (use the key icon on the left to input them):

- `METACULUS_TOKEN`: you can find your Metaculus token under your bot's user settings page: https://www.metaculus.com/accounts/settings/, or on the bot registration page where you created the account: https://www.metaculus.com/aib/
- `OPENAPI_API_KEY`: get one from OpenAIs page: https://platform.openai.com/settings/profile?tab=api-keys
- `PERPLEXITY_API_KEY` - used to search up-to-date information about the question. Get one from https://www.perplexity.ai/settings/api
- `ASKNEWS_CLIENT_ID`, `ASKNEWS_SECRET`

In [3]:
from omegaconf import OmegaConf
token_fn = "tokens.yaml"

In [4]:
config = OmegaConf.load(token_fn)

def pr(tokens):
    print(OmegaConf.to_yaml(config))

### LLM and Metaculus Interaction

This section sets up some simple helper code you can use to get data about forecasting questions and to submit a prediction

In [5]:
import datetime
import json
import os
import requests
import re
from openai import OpenAI
from tqdm import tqdm

In [6]:
AUTH_HEADERS = {"headers": {"Authorization": f"Token {config.METACULUS_TOKEN}"}}
API_BASE_URL = "https://www.metaculus.com/api2"
WARMUP_TOURNAMENT_ID = 3349
SUBMIT_PREDICTION = True

def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

def post_question_comment(question_id, comment_text):
    """
    Post a comment on the question page as the bot user.
    """

    response = requests.post(
        f"{API_BASE_URL}/comments/",
        json={
            "comment_text": comment_text,
            "submit_type": "N",
            "include_latest_prediction": True,
            "question": question_id,
        },
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Comment posted for ", question_id)

def post_question_prediction(question_id, prediction_percentage):
    """
    Post a prediction value (between 1 and 100) on the question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/predict/"
    response = requests.post(
        url,
        json={"prediction": float(prediction_percentage) / 100},
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Prediction posted for ", question_id)


def get_question_details(question_id):
    """
    Get all details about a specific question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/"
    response = requests.get(
        url,
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    return json.loads(response.content)

def list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=1000):
    """
    List (all details) {count} questions from the {tournament_id}
    """
    url_qparams = {
        "limit": count,
        "offset": offset,
        "has_group": "false",
        "order_by": "-activity",
        "forecast_type": "binary",
        "project": tournament_id,
        "status": "open",
        "type": "forecast",
        "include_description": "true",
    }
    url = f"{API_BASE_URL}/questions/"
    response = requests.get(url, **AUTH_HEADERS, params=url_qparams)
    response.raise_for_status()
    data = json.loads(response.content)
    return data

### IFP

In [7]:
class IFP:

    def __init__(self, question_id):
        self.question_id = question_id
        self.question_details = get_question_details(self.question_id)
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")   
        self.title = self.question_details["title"]
        self.resolution_criteria = self.question_details["resolution_criteria"]
        self.background = self.question_details["description"]
        self.fine_print = self.question_details["fine_print"]

    def report(self):
        rpt = f"""
The future event is described by this question: [ {self.title} ]
The resolution criteria are: [ {self.resolution_criteria} ]
The background is: [ {self.background} ]"""
        if self.fine_print:
            rpt += f"""
The fine print is: [ {self.fine_print} ]"""
        return rpt

### LLMs

### In general

In [8]:
class LLM:
    def __init__(self, system_role):
        self.messages = [{"role": "system", "content": system_role}]

    def chat(self, query):
        self.messages.append({"role": "user", "content": query})
        text = self.message()
        self.messages.append({"role": "assistant", "content": text})
        return text

### MetaAI

https://pypi.org/project/meta-ai-api/1.0.6/

In [9]:
 from meta_ai_api import MetaAI

### HuggingChat

https://pypi.org/project/hugchat/

In [10]:
from hugchat import hugchat
from hugchat.login import Login

### Anthropic

### AskNews

https://colab.research.google.com/drive/1tc383HraMZOiyfKFF1EXAtlTYbsuv3Q5?usp=sharing

In [11]:
from asknews_sdk import AskNewsSDK

### Perplexity

In [12]:
class Perplexity(LLM):
    def message(self):
        url = "https://api.perplexity.ai/chat/completions"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {config.PERPLEXITY_API_KEY}",
            "content-type": "application/json"  }
        payload = {"model": config.PERPLEXITY_MODEL, "messages": self.messages }
        response = requests.post(url=url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

### OpenAI

TBD: Metaculus credentials

In [13]:
class ChatGPT(LLM):
    def __init__(self, system_role):
        super().__init__(system_role)
        self.client = OpenAI(api_key=config.OPENAI_API_KEY)

    def message(self):
        chat_completion = self.client.chat.completions.create(
            model=config.OPENAI_MODEL,
            messages= self.messages)
        return chat_completion.choices[0].message.content

### Test questions

### Agents

In [14]:
class Agent:
    def __init__(self, system_role, llm):
        self.llm = llm(system_role)

    def chat(self, prompt):
        return self.llm.chat(prompt)

## Rate Analyzer(TBD: ChatGPT is better)

#### Researcher

In [15]:
class Researcher(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are an open source intelligence analyst.
You summarize news related to questions about events.
A question about an event is formatted as |id|question|criteria|background|fineprint|.
You will find on the web and report any reliable information you can gather about the question.
Do not make an assessment of probability.
Do not repeat information provided to you already in the prompt."""
        super().__init__(self.system_role, llm)

    def recherche(self, ifp):
        prompt = f"|{ifp.question_id}|{ifp.title}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.R = self.chat(prompt)
        ifp.news = self.R
        print(ifp.question_id, ifp.title)
        print(self.R)

    def research(self, ifps):
        for ifp in tqdm(ifps):
            self.recherche(ifps[ifp])

### Question relator

In [16]:
class QuestionRelator(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are prompted with list of forecasting questions, each with an id and a title.
Label each question with an underlying event.
If the questions are for the same event, use the same name for the underlying event.
Please return as separate lines formatted as |event|id|title|, do not add any other formatting.
"""
        super().__init__(self.system_role, llm)

    def relate(self, ifps):
        prompt = '\n'.join([f"{ifp.question_id}: {ifp.title}" for ifp in ifps.values()])
        KL = self.chat(prompt)
        K1 = [x.split('|') for x in KL.split('\n')]
        K2 = [(int(id),event) for _,event,id,_,_ in K1]
        for id,event in K2:
            ifps[id].event = event
            print(id, event)

#### Superforecaster

In [17]:
class Superforecaster(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are a superforecaster.  
You assign a probability to questions about events.
Questions are given as separate groups of lines formatted as |FORECAST|event|id|question|news|criteria|background|fineprint|.
Groups are separated by '^^^'.
Questions which are about the same event should be assigned consistent probabilities.
Reply to questions with |ASSESSMENT|id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and rationale is your reasoning for the forecast.
Separate each question with '^^^'.
Do not add any additional headings or group labels or other formatting.
After your initial forecast you may receive feedback of form |CRITIC|id|feedback|.
Reply to each feedback with |id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and 
and rationale is a revised assessment which may be adjusted from a prior assessment due the feedback unless the feedback is "I concur".
"""
        super().__init__(self.system_role, llm)

    def forecast(self, ifps):
        prompt = '^^^'.join([f"FORECAST|{ifp.event}|{ifp.question_id}|{ifp.news}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|" for ifp in ifps.values()])
        self.F0 = self.chat(prompt)
        self.F1 = [x.strip().replace('\n', '') for x in self.F0.split('^^^')]
        self.F2 = [x.split('|') for x in self.F1] 
        self.F3 = [[x for x in y if x] for y in self.F2]
        self.F4 = [(int(id),int(forecast),rationale) for _, id, forecast, rationale in self.F3]
        for id, forecast, rationale in self.F4:
            ifps[id].forecast = forecast
            ifps[id].rationale = rationale
            print(id, forecast, rationale)

    def reassess(self, ifp):
        prompt = f"|CRITIC|{ifp.question_id}|{ifp.feedback}|"
        self.R0 = self.chat(prompt)
        id,fcst,rationale = [x for x in self.R0.strip().split('|') if x]
        id = int(id)
        fcst = int(fcst)
        ifps[id].forecast = fcst
        ifps[id].rationale = rationale
        print(id, fcst, rationale)

#### Critic

In [18]:
class Critic (Agent):
    def __init__(self, llm):

        self.system_role = f"""
You a very smart and worldly person reviewing a superforecaster's assignment of probabilities to events.
You will receive an event with probabilities given as |event|id|question|zz|rationale|news|criteria|background|fineprint|.
zz is an integer probability from 1 to 99 and rationale is the student's logic for assigning probability of zz.
You will reply with a line |id|feedback| where feedback is "I concur" if you see no problem with the rationale and zz otherwise presents possible problems with the rationale and zz.
"""
        super().__init__(self.system_role, llm)

    def feedback(self, ifp):
        prompt = f"|{ifp.event}|{ifp.question_id}|{ifp.title}|{ifp.forecast}|{ifp.rationale}|{ifp.news}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.fb = self.chat(prompt)
        self.fb1 = self.fb.split('|')
        self.fb2 = [x for x in self.fb1 if x]
        try:
            id,feedback = self.fb2
            ifps[int(id)].feedback = feedback
            print(id, feedback)
        except:
            print('problem', self.fb2)
            ifps[int(id)].feedback = 'I concur'

## Summarizer

Add an agent to summarize the back and forth between critic and forecaster into a single cogent rationale that incorporates all that was discussed.

### Forecasting process

In [19]:
def forecasting(ifps):
    max_tries = 4
    
    analyst = Researcher(ChatGPT)
    analyst.research(ifps)
    
    qr = QuestionRelator(ChatGPT)
    qr.relate(ifps)
    
    sf = Superforecaster(ChatGPT)
    sf.forecast(ifps)
    
    for ifp in ifps.values():
        print("Refining", ifp.question_id)
        ifp.feedback = ''
        critic = Critic(Perplexity)
        for i in range(max_tries):
            print("Pass", i, "of", max_tries, "on", ifp.question_id)
            if 'I concur' in ifp.feedback:
                break
            critic.feedback(ifp)
            if 'I concur' in ifp.feedback:
                break
            sf.reassess(ifp)
        print("===============================================")

    for ifp in ifps.values():
        print(ifp.question_id, ifp.title)
        print("Forecast", ifp.forecast)
        print("Rationale", ifp.rationale, '\n')

In [20]:
def upload(ifp):
    post_question_prediction(ifp.question_id, ifp.forecast)
    post_question_comment(ifp.question_id, ifp.rationale)

In [21]:
def uploads(ifps):
    for ifp in ifps.values():
        upload(ifp)

## Daily forecast

### Get IFP ids

In [22]:
ifps = list_questions()['results']
today_ids = list(sorted([x['id'] for x in ifps]))
# today_ids = [25876, 25877, 25875, 25873, 25871, 25878, 25874, 25872] # 08JUL24
# today_ids = [26006, 25936, 25935, 25934, 25933, 26004, 26005] # 09JUL24
# today_ids = [25955, 25956, 25957, 25960, 25959, 25954, 25953, 25952, 25958] # 10JUL24
# today_ids = [26019, 26018, 26017, 26020, 26022, 26021, 26023, 26024] # 11JUL24
# today_ids = [26095, 26096, 26097, 26098, 26099, 26100, 26101, 26102] # 12JUL24
# today_ids = [26133, 26134, 26138, 26139, 26140, 26157, 26158, 26159] # 15JUL24
# today_ids = [26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196] # 16JUL24
# today_ids = [26210, 26211, 26212, 26213, 26214, 26215, 26216] # 17JUL24
# today_ids = [26232, 26233, 26234, 26235, 26236] # 18JUL24
# today_ids = [26302, 26303, 26304, 26305, 26306, 26307] # 19JUL24
# today_ids = [26387, 26388, 26389, 26390, 26391, 26392] # 22JUL24
# today_ids = [26404, 26405, 26406, 26407, 26408] # 23JUL24
# today_ids = [26550, 26551, 26552, 26553, 26554, 26555] # 24JUL24
# today_ids = [26568, 26569, 26570, 26571, 26572, 26573, 26574, 26575, 26576, 26577] # 25JUL24
# today_ids = [26638, 26639, 26640, 26641, 26642, 26643, 26644, 26645, 26646] # 26JUL24
# today_ids = [26665, 26666, 26667, 26668, 26669, 26670, 26671, 26683] # 29JUL24
# today_ids = [26700, 26701, 26702, 26703, 26704, 26705, 26706] # 30JUL24
# today_ids = [26816, 26817, 26818, 26819, 26820, 26821, 26844] # 31JUL24
# today_ids = 

In [24]:
today_ids = [26771, 26772, 26773, 26774, 26775, 26776, 26777, 26778, 26779, 26780, 26781]

## Forecast

In [25]:
ifps = {id: IFP(id) for id in today_ids}

In [26]:
forecasting(ifps)

  9%|████                                        | 1/11 [00:01<00:18,  1.88s/it]

26771 Will Fabiano Caruana win the 2024 Grand Chess Tour?
As of now, the official standings on the Grand Chess Tour website list Fabiano Caruana leading with 22.25 points, with two events still to be held: the 2024 Saint Louis Rapid & Blitz and the 2024 Sinquefield Cup. Caruana's performance in these remaining tournaments will determine if he wins the Grand Chess Tour. You may follow real-time updates on the official Grand Chess Tour site for the most accurate and up-to-date information on the standings and performances.


 18%|████████                                    | 2/11 [00:08<00:43,  4.87s/it]

26772 Will Alireza Firouzja win the 2024 Grand Chess Tour?
As of now, according to the official standings on the Grand Chess Tour website, Alireza Firouzja is in second place with 17.58 points, behind Fabiano Caruana who leads with 22.25 points. There are still two events remaining: the 2024 Saint Louis Rapid & Blitz and the 2024 Sinquefield Cup. Firouzja's performance in these upcoming tournaments will determine if he can overtake Caruana and win the Grand Chess Tour. You should regularly check the Grand Chess Tour site and other credible sources for real-time updates on standings and results.


 27%|████████████                                | 3/11 [00:10<00:28,  3.58s/it]

...

 91%|███████████████████████████████████████    | 10/11 [00:39<00:04,  4.01s/it]

26780 Will Bitcoin reach a new all-time high before October 1, 2024?
To determine if Bitcoin reaches a new all-time high (ATH) before October 1, 2024, you will need to regularly monitor the price listed on CoinMarketCap. The current ATH price as per CoinMarketCap is $73,750.07, recorded on March 14, 2024. The new ATH must exceed this price and occur between July 30 and October 1, 2024. 

**Key Points to Monitor:**
- **Current ATH:** $73,750.07 (as of March 14, 2024).
- **Date of potential new ATH:** Between July 30 and October 1, 2024.
- **Price Source:** CoinMarketCap (or an alternative like CoinGecko if data is ambiguous or unavailable on the primary site).

### Monitoring Steps:
1. Regularly check the Bitcoin price on [CoinMarketCap](https://coinmarketcap.com/currencies/bitcoin/).
2. Compare the current price with the existing ATH of $73,750.07.
3. Record any new ATHs that occur between July 30 and October 1, 2024.

If Bitcoin’s price reaches a new ATH above $73,750.07 in the specif

100%|███████████████████████████████████████████| 11/11 [00:45<00:00,  4.10s/it]

26781 Will William Ruto cease to be President of Kenya before October 1, 2024?
As of now, William Ruto remains the President of Kenya despite the serious political unrest and significant actions such as the dismissal and partial reinstatement of his cabinet, the resignation of the police chief, and ongoing nationwide protests calling for his resignation.

### Key Points to Monitor:
1. **Current Status:** William Ruto is still the President of Kenya.
2. **Criteria for Resolution as Yes:**
    - Resignation
    - Impeachment
    - Losing an election
    - Removal from office via a coup
    - Any other reason leading to ceasing to be President before October 1, 2024
3. **Compliance with Fine Print:** If Ruto is temporarily replaced but resumes duties within 30 days, it does not count. If he resumes duties after more than 30 days (completed before October 1, 2024), it resolves as Yes.
4. **Credible Sources:** Information from major news outlets and official listings like the [UN Heads of S

26771 2024 Grand Chess Tour
26772 2024 Grand Chess Tour
26773 2024 Grand Chess Tour
26774 2024 Grand Chess Tour
26775 2024 Grand Chess Tour
26776 Seattle-Tacoma-Bellevue WA Air Quality
26777 Seattle-Tacoma-Bellevue WA Air Quality
26778 Seattle-Tacoma-Bellevue WA Air Quality
26779 Seattle-Tacoma-Bellevue WA Air Quality
26780 Bitcoin Price
26781 William Ruto Presidency
26771 60 Fabiano Caruana holds a significant lead and has shown strong performance in the tour so far. While there are still two tournaments left, his point advantage makes it relatively more likely for him to secure the overall win despite potential strong performances by rivals.
26772 30 Alireza Firouzja is currently behind Caruana and while he possesses the skill to potentially outperform Caruana in the remaining tournaments, the point gap suggests it is less likely.
26773 20 Praggnanandhaa Rameshbabu is further behind in the standings compared to Caruana and Firouzja, making it less probable for him to win the overall 

## Upload

In [27]:
uploads(ifps)

Prediction posted for  26771
Comment posted for  26771
Prediction posted for  26772
Comment posted for  26772
Prediction posted for  26773
Comment posted for  26773
Prediction posted for  26774
Comment posted for  26774
Prediction posted for  26775
Comment posted for  26775
Prediction posted for  26776
Comment posted for  26776
Prediction posted for  26777
Comment posted for  26777
Prediction posted for  26778
Comment posted for  26778
Prediction posted for  26779
Comment posted for  26779
Prediction posted for  26780
Comment posted for  26780
Prediction posted for  26781
Comment posted for  26781


In [28]:
len(today_ids)

11